In [1]:
from transformers import AutoTokenizer, AutoConfig, GPTJForCausalLM

In [2]:
import json

with open('/home/ubuntu/server2/ssd3/common-crawl/common-crawl-qa.json') as fopen:
    common_crawl = json.load(fopen)

In [3]:
common_crawl[100]

['Contact us 1300 30 4227 | English Bahasa Malaysia 中文 Sell Sort Search Home Buy Cars for sale Used Cars New Cars Recon Cars Kereta Murah Hot Deals CARSOME Certified Certified Pre-owned BMW Premium Selection Sell Sell Your Car Create Ad FREE How to sell your car New Cars New Car Deals NEW New Car Price List Finance Car Loan Car Insurance Car Insurance Comparison Extended Warranty News All News Auto News Insights Reviews Buying Guide Electric Vehicle (EV) Photos Videos Events Event Services Hire us! Become a Partner Upcoming Events Car of The Year 2022 Past Events View All Past Events Sign In Login Login as Dealer / Agent or Register Register Register as Dealer / Agent Sell your car EN EN English BM Bahasa Malaysia CN 中文 Home Buy Cars for sale Used Cars New Cars Recon Cars Kereta Murah Hot Deals CARSOME Certified Certified Pre-owned BMW Premium Selection Sell Sell Your Car Create Ad FREE How to sell your car New Cars New Car Deals NEW New Car Price List Finance Car Loan Car Insurance Ca

In [4]:
json.loads(common_crawl[100][1])

{'soalan': [{'soalan': 'Apakah yang sedang diumumkan oleh BMW Malaysia?',
   'jawapan': 'BMW Malaysia sedang mengumumkan iX Electric SUV.'},
  {'soalan': 'Apakah jenis kereta yang diumumkan oleh BMW Malaysia?',
   'jawapan': 'BMW Malaysia mengumumkan iX Electric SUV.'},
  {'soalan': 'Apakah kelebihan kereta elektrik iX Electric SUV?',
   'jawapan': 'iX Electric SUV adalah kereta elektrik yang mesra alam dan mempunyai prestasi yang baik.'}]}

In [5]:
template = {
    "description": "Template used by Alpaca-LoRA.",
    "prompt_input": "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n",
    "prompt_no_input": "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Response:\n",
    "response_split": "### Response:"    
}

In [6]:
import random
from tqdm import tqdm

In [7]:
import malaya

model = malaya.augmentation.abstractive.huggingface(model = 'mesolitica/finetune-noisy-translation-t5-small-bahasa-cased-v3', force_check = False)

In [8]:
_ = model.cuda()

In [9]:
model.generate(['rakyat memang tak suka awak pun'],
              max_length = 100, do_sample=True, top_k=100, top_p=0.95, temperature=0.7,
              num_return_sequences=5)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


['Rakyat mmg tak suka kau pun',
 'Rakyat mmg tak suka ko pun',
 'rakyat mng x suka kau pun',
 'rakyat mmg tak suka ko pon',
 'rakyat mmg tak suka kau pon']

In [10]:
data_common_crawl = []
for d in tqdm(common_crawl):
    try:
        t = d[0]
        a = d[1]
        j_a = json.loads(d[1])
        q = t
        
        for r in j_a['soalan']:
            if random.random() > 0.5:
                generated = model.generate(
                    [r['soalan']],
                    max_length = 256, 
                    do_sample=True, top_k=100, top_p=0.95, temperature=0.7,
                    num_return_sequences=5,
                )
                soalan = random.choice(generated)
            else:
                soalan = r['soalan']
                
            if random.random() > 0.5:
                soalan = soalan.lower()
            
            if random.random() > 0.7:
                q_ = f"paragraph `{q}`, berdasarkan paragraph, jawab soalan `{soalan}`"
            else:
                q_ = f"paragraph `{q}`, {soalan}"
            
            res = template["prompt_no_input"].format(instruction=q_)
            res = f"{res}{r['jawapan']}"
            data_common_crawl.append(res)
        
        k = '({"soalan": [{`soalan`, `jawapan`}]})'
        p = f"""
paragraph, `{t}`
hasilkan 3 soalan dan jawapan khusus berdasarkan perenggan itu, kembalikan sebagai struktur JSON {k}
        """
        res = template["prompt_no_input"].format(instruction=p)
        res = f"{res}{a}"
        data_common_crawl.append(res)

    except Exception as e:
        print(e)
        pass

  0%|▎                                                                                             | 201/55900 [00:38<1:04:55, 14.30it/s]

Extra data: line 18 column 1 (char 829)
Extra data: line 18 column 1 (char 790)
Extra data: line 18 column 1 (char 617)
Extra data: line 18 column 1 (char 665)
Extra data: line 18 column 1 (char 752)


  1%|▍                                                                                             | 289/55900 [00:53<1:21:16, 11.40it/s]

Extra data: line 18 column 1 (char 616)
Extra data: line 18 column 1 (char 677)


  1%|▍                                                                                             | 293/55900 [00:54<1:33:47,  9.88it/s]

...

100%|███████████████████████████████████████████████████████████████████████████████████████████▉| 55895/55900 [2:43:03<00:00,  6.26it/s]

Extra data: line 18 column 1 (char 659)


100%|███████████████████████████████████████████████████████████████████████████████████████████▉| 55898/55900 [2:43:03<00:00,  8.84it/s]

Extra data: line 18 column 1 (char 853)
Extra data: line 18 column 1 (char 947)


100%|████████████████████████████████████████████████████████████████████████████████████████████| 55900/55900 [2:43:03<00:00,  5.71it/s]


In [11]:
len(data_common_crawl)

212526

In [13]:
data_common_crawl[0]

'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nparagraph `Google Map) MAZLAN MOKHTAR JENGKA Jerantut (Klik nama untuk WhatsApp, klik lokasi untuk Google Map) MOHD NAJID JERANTUT Kuantan (Klik nama untuk WhatsApp, klik lokasi untuk Google Map) FAZIRA JALAN TELUK SISEK FAZIRAH ZUKI LORONG CHENGAL LEMPONG NORAZMA KUANTAN AZIZAH KUANTAN NANAD AZRI TAMAN BUKIT RANGIN NURULHANA SUNGAI ISAP ROHANI MUHAMAD JALAN AIR PUTIH (1 - 5) CIK TA JALAN INDUSTRI TANAH PUTIH BARU SAADIAH KAMPUNG BALOK SHARIFAH KUANTAN SITI KHADIJAH KAMPUNG TIRAM SITI NORHAMIZAH TAMAN SERI MAHKOTA AMAN SITI SHARINA KUANTAN ZALIFAH AHMAD BANDAR INDERA MAHKOTA Maran (Klik nama untuk WhatsApp, klik lokasi untuk Google Map) MOHD FADELY BUKIT TAJAU Muadzam Shah (Klik nama untuk WhatsApp, klik lokasi untuk Google Map) HASAZALI PAHANG Pekan (Klik nama untuk WhatsApp, klik lokasi untuk Google Map) FASEEHA KAHAR PEKAN Raub (Klik nama untuk WhatsApp, kl

In [ ]:
from transformers import AutoTokenizer
base_model='EleutherAI/pythia-6.9b'
tokenizer = AutoTokenizer.from_pretrained(base_model)

In [ ]:
len(tokenizer.tokenize(data_common_crawl[0]))

In [14]:
with open('prepared-common-crawl.json', 'w') as fopen:
    json.dump(data_common_crawl, fopen)

In [15]:
!ls -lha prepared-common-crawl.json

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
-rw-r--r-- 1 husein-comel husein-comel 694M Apr  26 20:15 prepared-common-crawl.json
